In [1]:
import pymongo
import pandas as pd
import itertools
import csv
import tqdm

OUTFILE = 'graduates.csv'

In [2]:
client = pymongo.MongoClient()
db = client['uni']

In [3]:
def get_graduates(vpo1_collection):
    return vpo1_collection.aggregate(
        pipeline=[
    {
        '$match': {
            'section': '2.1.3'
        }
    }, {
        '$project': {
            'columns': 0
        }
    }, {
        '$unwind': {
            'path': '$rows', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$match': {
            'rows.values.2.colname': 'Код классификатора', 
            'rows.values.2.value': 1
        }
    }, {
        '$unwind': {
            'path': '$rows.values', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$match': {
            'rows.values.colname': {
                '$regex': 'Выпуск фактический.*'
            }
        }
    }, {
        '$project': {
            'region': 1, 
            'funded_by': 1, 
            'time_involvement': 1, 
            # 'section': 1, 
            # 'section_name': 1, 
            'colname': '$rows.values.colname', 
            'group': '$rows.group', 
            'program': '$rows.name', 
            'value': '$rows.values.value'
        }
    }
])

In [4]:
fieldnames = list(next(get_graduates(db.vpo1)).keys())
fieldnames.remove('_id')
# fieldnames.remove('group')
fieldnames

['region',
 'funded_by',
 'time_involvement',
 'colname',
 'group',
 'program',
 'value']

In [5]:
with open(OUTFILE, 'w', newline='') as f:
    f.truncate(0)
    writer = csv.DictWriter(f, fieldnames=fieldnames, dialect='unix')
    writer.writeheader()
    for record in tqdm.tqdm(get_graduates(db.vpo1)):
        # record['program'] = '{}. {}'.format(record['group'], record['program'])
        for k in list(record.keys()):
            if k not in fieldnames:
                del record[k]
        writer.writerow(record)

43300it [00:02, 16515.52it/s]


In [6]:
df = pd.read_csv(OUTFILE)
df

,region,funded_by,time_involvement,colname,group,program,value
0,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1,Информатика и вычислительная техника,44.0
1,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1,Информатика и вычислительная техника,0.0
2,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1,Психология,1.0
3,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1,Психология,0.0
4,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1,Экономика,371.0
5,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1,Экономика,0.0
6,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1,Менеджмент,495.0
7,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1,Менеджмент,0.0
8,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1,Государственное и муниципальное управление,98.0
9,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1,Государственное и муниципальное управление,0.0


In [7]:
df[df['program'] == '1. Математика'].head()

,region,funded_by,time_involvement,colname,group,program,value


In [8]:
df['colname'].unique()

array(['Выпуск фактический с 01.10. прошлого года по 30.09. текущего года всего (сумма гр. 10, 12, 13, 15)',
       'Выпуск фактический с 01.10. прошлого года по 30.09. текущего года из них лица с ОВЗ, инвалиды, детиинвалиды'],
      dtype=object)

In [9]:
df.program.unique()

array(['Информатика и вычислительная техника', 'Психология', 'Экономика',
       'Менеджмент', 'Государственное и муниципальное управление',
       'Бизнесинформатика', 'Юриспруденция', 'Зарубежное регионоведение',
       'Реклама и связи с общественностью', 'Журналистика', 'Туризм',
       'Педагогическое образование',
       'Психологопедагогическое образование', 'Лингвистика', 'Теология',
       'Экономическая безопасность', 'Таможенное дело',
       'Правовое обеспечение национальной безопасности',
       'Специальное (дефектологическое) образование',
       'Прикладная математика и информатика', 'Прикладная математика',
       'Математическое обеспечение и администрирование информационных систем',
       'Экология и природопользование', 'Архитектура',
       'Реконструкция и реставрация архитектурного наследия',
       'Дизайн архитектурной среды', 'Градостроительство',
       'Строительство', 'Информационные системы и технологии',
       'Прикладная информатика', 'Программная инж